# Plot the SV for one year and all zones

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import package.sql_manager as sm
from matplotlib.ticker import ScalarFormatter

sep = ['\\','/'][sys.platform=='linux']

savedir = paperdir + 'Pictures%sStorage value vs RTE%sPJM%s'%(sep,sep,sep)



zone_list = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']

market_type = '_dart'#da
RTEs = np.arange(100,20,-10)
SH = 10
loss=0
Colors = ['k', 'gray', 'r', 'g', 'b', 'm', 'y', 'c' ]
Pout=50
zones = zone_list[12:16]
years = np.arange(2010,2020,1)

plt.rc('text', usetex=True)
plt.close("all")
font_size = 16
fig, ax = plt.subplots(figsize=(16,12),nrows=2, ncols=2, gridspec_kw = {'wspace':0.1, 'hspace':0.15})
ax=ax.ravel()
xfmt = ScalarFormatter()
xfmt.set_powerlimits((-3,3))
    
for s,Zone in enumerate(zones):
    db='storage_value_PF_simple_PJM%s.db'%(['',market_type][market_type=='_dart'])
    table = sm.list_tables(db)[0]
    cols = [col[0] for col in sm.list_columns(db,table)]
    Storage = sm.get_data(cols, table, db)
    Storage['date_time']=Storage['date_time'].astype(int)
    
    for (r,RTE) in enumerate(RTEs):
        data = pd.DataFrame(columns=['year', 'SV'])
        storage = Storage[(Storage['zone']==Zone) &
                          (Storage['rte']==RTE) &
                          (Storage['sh']==SH) &
                          (Storage['loss']==loss)&
                          (Storage['Pin_max']==50)&
                          (Storage['Pout_max']==50)]
        
        storage = storage.sort_values('date_time', axis=0)
        ax[s].bar(storage['date_time'], storage['SV']/(Pout), width=0.4,
                       color=Colors[r], label= r'$ %d\%% $'%(RTEs[r]), edgecolor='k')

    ax[s].set_title(r'%s'
                      % (Zone), fontsize=font_size)

    ax[s].set_xticks(list(years))
    ax[s].yaxis.set_major_formatter(xfmt)
    if s in [0,2]:
        ax[s].set_ylabel(r'$ \textrm{SV} \; [\$\textrm{/MW-year}]$', fontsize=font_size)
        
    if s in [2,3]:
        ax[s].set_xlabel(r'$\textrm{Year}$', fontsize=font_size)
    
    if s in [0,1]:
        ax[s].tick_params(labelbottom=False)
    if s==3:
        lg = ax[s].legend(title=r'$\textrm{RTE}$', loc=(1.02,0),
                ncol=1, fontsize=font_size, edgecolor='k',
                handletextpad=0.5, columnspacing =1.37)
        lg.get_title().set_fontsize(font_size)

    ax[s].grid()
    ax[s].set_axisbelow(True)

plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)

plt.savefig(savedir + 'SV_RTE_%s_%s.png'%("_".join(zones).replace("/","-"), market_type),  dpi=600, bbox_inches = "tight")
plt.savefig(savedir + 'SV_RTE_%s_%s.svg'%("_".join(zones).replace("/","-"),market_type),  bbox_inches = "tight")
plt.show()

# AOI calculations PJM

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

from projdirs import paperdir
from datetime import timedelta
import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
import pdb
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
%matplotlib inline
plt.rc('text', usetex=True)
sep = ['\\','/'][sys.platform=='linux']

years=np.arange(2010,2020,1)
RTEs = np.arange(100,20,-10)
SPIKES = pd.DataFrame(columns=['year', 'spikes_vol'])

zone_list = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']

zones = zone_list
for z,Zone in enumerate(zones):
    lmps = gd.load_lmp_zone(Zone)
    for i,Year in enumerate(years):
        LMP = lmps[lmps.index.year==Year]
        c = LMP
        Q1 = c.quantile(0.25)
        Q3 = c.quantile(0.75)
        IQR = Q3-Q1
        c_trunc = c#[(c<(Q3+3*IQR)) | (c>(Q1-3*IQR))]
        
        for (r,RTE) in enumerate(RTEs):
            a=100/RTE
            mean_trunc = c_trunc.mean()
            spikes_vol = ((c[c>(mean_trunc*a)]-mean_trunc).sum() + abs(c[c<(mean_trunc*(1/a))]-mean_trunc).sum())
            SPIKES = SPIKES.append({'year':Year,
                                    'zone':Zone,
                                    'rte':RTE,
                                    'spikes_vol': spikes_vol}
                                       , ignore_index=True)
        
        SPIKES = SPIKES.sort_values('year', axis=0)
        
normaliser = SPIKES['spikes_vol'].max()

In [ ]:
# Plot the results  
plt.close("all")
font_size = 14
fig, ax = plt.subplots(figsize=(16,6),nrows=1, ncols=2,
                       gridspec_kw = {'wspace':0.1, 'hspace':0.15})
ax=ax.ravel()
Colors = ['k', 'gray', 'r', 'g', 'b', 'm', 'y', 'c' ]
Zones = zones[16:18]
for z,Zone in enumerate(Zones):
    for r,RTE in enumerate(RTEs):
        DATA = SPIKES[(SPIKES['rte']==RTE)&
                      (SPIKES['zone']==Zone)]
        ax[z].plot(DATA['year'], DATA['spikes_vol']/normaliser,'--o',
                   linewidth=1, color=Colors[r],
                   label= r'$ %d\%% $'%(RTEs[r]))
    ax[z].grid(which='both', axis='both')
    ax[z].text(0.5,1.05,r'$\textrm{%s}$'%(Zone),
                  fontsize=font_size, transform=ax[z].transAxes, ha='center')
    if z in [0]:
        ax[z].set_ylabel(r'$\textrm{Normalised AOI}$', fontsize= font_size)
    if z in [1]:
        ax[z].tick_params(labelleft = False)
#     if z in [0,1]:
#         ax[z].tick_params(labelbottom = False)
    ax[z].set_xlabel( r'$\textrm{Year}$', fontsize= font_size)
    ax[z].set_axisbelow(True)
    ax[z].set_xticks(list(years))
    ax[z].set_yticks(np.arange(0,1.1,0.1))
    ax[z].set_ylim([0,1.1])
    if z==3:
        lg = ax[z].legend(title=r'$\textrm{RTE}$', loc=(1.02,0),
                ncol=1, fontsize=font_size, edgecolor='k',
                handletextpad=0.5, columnspacing =1.37)
        lg.get_title().set_fontsize(font_size)
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)

savedir = paperdir + 'Pictures%sstorage value vs RTE%sspike volume%sPJM%s'%(sep,sep,sep,sep)
# plt.savefig(savedir + 'spike_volume_%s.png'%("_".join(Zones[0:-1]).replace("/","-")), dpi=600, bbox_inches = "tight")
plt.savefig(savedir + 'spike_volume_%s.svg'%("_".join(Zones[0:-1]).replace("/","-")), bbox_inches = "tight")

# Surf plot PJM prices for all zones

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
from projdirs import paperdir
from pandas.plotting import register_matplotlib_converters
plt.rc('text', usetex=True)
sep = ['\\','/'][sys.platform=='linux']

zone_list = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
lmp_type='rt'
font_size = 12
def f(x,y):
    if datetime.datetime.combine(x, y) in zonal_price.index:
        value = zonal_price.loc[(datetime.datetime.combine(x, y))]
        time = datetime.datetime.combine(x, y)
        
        return np.array([value,value][isinstance(value, list)])
    else:
        return np.array(20)

def ff(x):
    return x.hour*60+x.minute


fig = plt.figure(figsize=(16, 24.5))
fig.tight_layout()
fig.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1, wspace=0.0, hspace=0.01)

price_spot = gd.load_lmp_cal(2010)
years = np.arange(2010,2020,1)
for Zone in zone_list[0:1]:
    print('Zone=',Zone) 
    for y,Year in enumerate(years):
        price_spot = gd.load_lmp_cal(Year, lmp_type)
        zonal_price = price_spot[(price_spot['pnode_name']==Zone)].set_index('datetime_beginning_ept')['total_lmp_%s'%(lmp_type)]

        x1 = pd.date_range(zonal_price.index.date.min(),
                           zonal_price.index.date.max()-pd.Timedelta('1D'), freq='1D').date
        y1 = pd.date_range('00:00:00','22:00:00', freq='1H').time

        x2, y2 = np.meshgrid(x1,y1)

        g = np.vectorize(f)
        gg = np.vectorize(ff)
        z = g(x2,y2)
        # z[z<-50]=-50
        # z[z>300]=300

        ax = fig.add_subplot(5,2,y+1, projection='3d', proj_type = 'ortho')
        surf = ax.plot_surface(dates.date2num(x2), gg(y2), z, rstride=1, cstride = 1,
                       edgecolor='k', linewidth=0.2, cmap=cm.jet)

        x_tick_range = pd.date_range(zonal_price.index.date.min(),
                   zonal_price.index.date.max(), freq='1D').date
        ax.set_xlim(x_tick_range.min(), x_tick_range.max())
        ax.xaxis.set_tick_params(rotation=90 )

        ax.set_zticklabels(ax.axes.get_zticks(), horizontalalignment= 'left')
        ax.zaxis.set_major_formatter(FormatStrFormatter('%.0f'))

        ax.axes.set_yticks(np.arange(0,25*60,120))
        ax.axes.set_ylim([0,1450])
        ytick_range = ax.axes.get_yticks()
        m = len(ax.axes.get_yticks())
        ylabels = pd.date_range(start = '2000-01-01', end = '2000-01-02',  periods=m)
        ylabels = [datetime.datetime.strftime(Label, '%H:%M') for Label in ylabels]
        ax.set_yticklabels(ylabels,rotation=-90, va='top', ha='right')

        ax.set_zlabel(r'$\textrm{LMP }[\$/\textrm{MWh}]$', fontsize=font_size, labelpad=15)
        ax.axes.invert_xaxis()
        ax.xaxis.set_tick_params(labelsize=font_size)
        ax.yaxis.set_tick_params(labelsize=font_size)
        ax.zaxis.set_tick_params(labelsize=font_size)
#         if y==0:
#             ax.set_title('%s - %d' %(Zone,Year), {
#                 'fontsize': font_size,
#                 'fontweight' : 'bold',
#                 'verticalalignment': 'top'
#                 })

        ax.view_init(elev=30)
        ax.dist = 12

    # fig.subplots_adjust(left=0.2, right=1, bottom=0.5, top=1)
    savedir = paperdir + 'Pictures%sRRP annual%sSurfplots%sPJM%s'%(sep,sep,sep,sep)
#     plt.savefig(savedir + "LMP_surf_%s_%d_%d.png"
#                 %(Zone.replace('/','-'),years[0], years[-1])
#                 ,format='png', dpi=600, bbox_inches = "tight")

plt.show()





# Surfplot prices in BGE and SA

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
from projdirs import paperdir
from pandas.plotting import register_matplotlib_converters
import pdb
plt.rc('text', usetex=True)
sep = ['\\','/'][sys.platform=='linux']


PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']


regions = ['SA', 'BGE']
years = [2010,2019]



# choose the state and date here
font_size = 12

def f(x,y):
    if (datetime.datetime.combine(x, y)) in data.index:
        Value = np.array(data.loc[(datetime.datetime.combine(x, y))]['spot_price'])
    else:
        Value = np.array(data.loc[(datetime.datetime.combine(x, y))-
                                  pd.Timedelta('1H')]['spot_price'])   
    
    return(Value)

def ff(x):
    return x.hour*60+x.minute

# plt.close("all")
fig = plt.figure(figsize=(16,12))
fig.tight_layout()
fig.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1, wspace=0, hspace=0.01)

i = 0
for Region in regions:
      for Year in [2010,2019]:
            i+=1
            if Region in NEM:
                price_spot=pd.DataFrame(gd.load_rrp_cal_stamped(Year, Region))
                price_spot['year']=Year
                price_spot['spot_price']= gd.convert_curr(price_spot['spot_price'],Year)
                
                Freq = '30min'
                EoD = '23:30:00'
            else:
                price_spot=pd.DataFrame(gd.load_lmp_zone_year(Region,Year)).reset_index()
                price_spot = price_spot.rename(
                    columns={'datetime_beginning_ept':'date_time',
                             'total_lmp_da':'spot_price'})
                Freq = '1H'
                EoD = '23:00:00'
                
            price_spot['date_time'] = pd.to_datetime(price_spot['date_time'])
            data = price_spot.set_index('date_time')
            
            x1 = pd.date_range(data.index.date.min(),
                               data.index.date.max()-pd.Timedelta('1D'), freq='2D').date
            y1 = pd.date_range('00:00:00',EoD, freq=Freq).time
      
            x2, y2 = np.meshgrid(x1,y1)
            
            g = np.vectorize(f)
            gg = np.vectorize(ff)
            z = g(x2,y2)
            z[z<-50]=-50
            z[z>300]=300

            ax = fig.add_subplot(220+i, projection='3d', proj_type = 'ortho')
            
            
            surf = ax.plot_surface(dates.date2num(x2), gg(y2), z, rstride=1, cstride = 1,
                           edgecolor='k', linewidth=0.2, cmap=cm.jet)
                        
            x_tick_range = pd.date_range(data.index.date.min(),
                       data.index.date.max(), freq='1D').date
            ax.set_xlim(x_tick_range.min(), x_tick_range.max())
            ax.xaxis.set_tick_params(rotation=90 )
            
            ax.set_zticklabels(ax.axes.get_zticks(), horizontalalignment= 'left')
            ax.zaxis.set_major_formatter(FormatStrFormatter('%.0f'))
            
            ax.axes.set_yticks(np.arange(0,25*60,120))
            ax.axes.set_ylim([0,1450])
            ytick_range = ax.axes.get_yticks()
            m = len(ax.axes.get_yticks())
            ylabels = pd.date_range(start = '2000-01-01', end = '2000-01-02',  periods=m)
            ylabels = [datetime.datetime.strftime(Label, '%H:%M') for Label in ylabels]
            ax.set_yticklabels(ylabels,rotation=-90, va='top', ha='right')
                
            ax.set_zlabel(r'$\textrm{RRP }[\$/\textrm{MWh}]$', fontsize=font_size, labelpad=15)
            ax.axes.invert_xaxis()
            ax.xaxis.set_tick_params(labelsize=font_size)
            ax.yaxis.set_tick_params(labelsize=font_size)
            ax.zaxis.set_tick_params(labelsize=font_size)
            ax.set_title('%s - %d' %(Region,Year), {
                    'fontsize': font_size,
                    'fontweight' : 'bold',
                    'verticalalignment': 'top'
                    })
                
            ax.view_init(elev=30)
            ax.dist = 13
            
# fig.subplots_adjust(left=1, right=1.2, bottom=0, top=1)



# plt.show()
savedir = paperdir + 'Pictures%sRRP annual%sSurfplots%s'%(sep,sep,sep)
plt.savefig(savedir + "RRP_surf.png", format='png', dpi=600,  bbox_inches = "tight")
plt.savefig(savedir + "RRP_surf.svg", format='svg', bbox_inches = "tight")


# Plot PJM capacity prices for all the zones from 2010 until 2020

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import paperdir
from datetime import timedelta
import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
import pdb
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
%matplotlib inline
plt.rc('text', usetex=True)
sep = ['\\','/'][sys.platform=='linux']

zone_list = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']

CAP_prices = pd.DataFrame()
RES = pd.DataFrame()
years = np.arange(2010,2020,1)
CAP_prices['year'] = np.arange(2010,2020,1)
for Zone in zone_list[2:]:
    RES['%s'%(Zone)] = CAP_prices.apply(lambda x: gd.get_capa_price(x,Zone), axis=1)
    
CAP_prices = pd.concat([CAP_prices,RES],axis=1)


from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = go.Figure()
for zone in CAP_prices.columns[1:]:
    fig.add_trace(go.Scatter(x=CAP_prices['year'], y=CAP_prices[zone],
                    name=zone))

fig.update_layout(
    title={'text': "PJM capacpity prices",
                   'y':0.9,
                   'x':0.5,
                   'xanchor': 'center',
                   'yanchor': 'top'},
    xaxis_title="Year",
    yaxis_title="Capacity price [$/MW-day]",
    
)

fig.show()

# SV in PJM capacity market
This code calculates the SV for arbitrage in the energy market.
This code is not superseded by ```Arbitrage into DA and RT markets```

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
from projdirs import paperdir
from datetime import timedelta
import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
import package.optimisation as op
%matplotlib inline
plt.rc('text', usetex=True)
sep = ['\\','/'][sys.platform=='linux']

RTE = 90
Pout_max = 50 #MW
Pin_max = 100 #MW
Pin_min = 0
SH = 10.5 #hrs
Q_init = 1
Q_end = 1
Year = 2018
Zone = 'DPL'
capa_price = gd.get_capa_price(Year, Zone) #$/MW-day

LMP_rt = round(gd.load_lmp_zone_year(Zone,Year,lmp_type='rt', Year_type='fy'),2)
DATA = LMP_rt.reset_index()
DATA.columns = ['date_time','lmp_rt']

# get the index of the beginning of the day as an integer for mizinc constratint
Q_constraint = (DATA[DATA['date_time'].dt.time == datetime.time(hour=0)].index+1).tolist()


# get the top 1% of high RT prices
price_spikes = LMP_rt.sort_values()[round(-len(LMP_rt)/100):]

# take N of price_spikes as emergyenecy hours
N=30
emer_hrs = price_spikes.sample(N)
DATA['Pout']=0
DATA = DATA.set_index('date_time')
DATA.loc[emer_hrs.index,'Pout']=1

c_rt = LMP_rt.tolist()
Pout_rt_string = str((Pout_max*DATA['Pout']).tolist())

simparams = op.load_simparams()
simparams['eta_in'] = 1
simparams['eta_out'] = RTE/100
simparams['Pin_max'] = Pin_max
simparams['Pout_max'] = Pout_max
simparams['Pin_min'] = Pin_min
simparams['dt'] = 1
simparams['q_max'] = SH * Pout_max / (RTE/100)
simparams['Pout_rt_string'] = Pout_rt_string
simparams['c_rt'] = c_rt
simparams['N'] = len(LMP_rt)
simparams['q_constraint'] = Q_constraint
simparams['SH'] = SH
simparams['q_min'] = 0
simparams['Pin_min'] = 0
simparams['loss'] = 0.01
del simparams['Pout_min_frac']
del simparams['Q_min_frac']
del simparams['Q_init_frac']
del simparams['Pin_min_frac']
del simparams['Pout_max']

from package.make_dzn_file import make_dzn_file_PJM_capacity
ID = Zone+str(Year)
make_dzn_file_PJM_capacity(ID, **simparams)
results = op.optimise_PJM_capacity(Zone+str(Year),simparams)

SV_energy = results['obj']
SV_capacity = capa_price*365*Pout_max
print('Energy revenue = ', SV_energy)
print('Capacity revenue = ', SV_capacity)

# print('SV for arbitrage = ', obj.item(0))
# capa_price = gd.get_capa_price(Year, Zone)
# print('Capacity market price = ', capa_price )
# print('Total SV = ', obj.item(0)+365*Pout_max*capa_price)


# Plot capacity market optimisation results
This code plots the results obtained from the previous cell.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df = pd.DataFrame()
df['date_time'] = LMP_rt.index
df['Q'] = results['Q'][:-1]
df['Pin_rt'] = results['Pin_rt']
df['Pout_rt'] = results['Pout_rt']
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['date_time'], y=df['Q'],
                         mode='lines',
                         name='Q',
                         line_shape='vh'))
fig.add_trace(go.Scatter(x=df['date_time'], y=df['Pout_rt'],
                         mode='lines',
                         name='Pout_rt',
                         line_shape='vh'), secondary_y=True)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

# Analyse behaviour of the storage in the PJM arbitrage

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

from projdirs import paperdir
from datetime import timedelta
import datetime
import numpy as np
import pdb
import pandas as pd
import package.get_NEM_data as gd
import package.optimisation as op
import multiprocessing as mp
import package.sql_manager as sm
from package.optimisation import optimise_PJM_parallel

from plotly.subplots import make_subplots
import plotly.graph_objects as go
Pin_max = 50
Pout_max = 50
SH =10
RTE = 40
Loss=0
Market_type = 'DA+RT'
Zone = 'BGE'
Year = 2015

results = optimise_PJM_parallel(Zone,Year, Pin_max, Pout_max,
                                SH, RTE, Loss, Market_type, details='on')


In [ ]:
lmps_da = gd.load_lmp_zone_year(Zone, Year, lmp_type='da', Year_type='cal')
lmps_rt = gd.load_lmp_zone_year(Zone, Year, lmp_type='rt', Year_type='cal')

df = pd.DataFrame()
df['date_time'] = lmps_da.index
df['Q'] = results['Q'][1:]
df['Pout_rt'] = results['Pout_rt']
df['Pin_rt'] = results['Pin_rt']
df['Pout_da'] = results['Pout_da']
df['Pin_da'] = results['Pin_da']


fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['date_time'], y=df['Q'],
                    mode='lines',
                    name='Q'))

fig.add_trace(go.Scatter(x=df['date_time'], y=df['Pin_rt'],
                    mode='lines',
                    name='Pin_rt', line_shape='vh'),
             secondary_y=True)

fig.add_trace(go.Scatter(x=df['date_time'], y=df['Pout_da'],
                    mode='lines',
                    name='Pout_da', line_shape='vh'),
             secondary_y=True)

fig.add_trace(go.Scatter(x=df['date_time'], y=df['Pout_rt'],
                    mode='lines',
                    name='Pout_rt', line_shape='vh'),
             secondary_y=True)

fig.add_trace(go.Scatter(x=df['date_time'], y=df['Pin_da'],
                    mode='lines',
                    name='Pin_da', line_shape='vh'),
             secondary_y=True)

lmps_da = lmps_da.reset_index()
lmps_rt = lmps_rt.reset_index()
fig.add_trace(go.Scatter(x=lmps_da['datetime_beginning_ept'],
                         y=lmps_da['total_lmp_da'],
                         name = 'DA prices',
                         yaxis = 'y4',
                         line_shape='vh'
              ))
              
fig.add_trace(go.Scatter(x=lmps_rt['datetime_beginning_ept'],
                         y=lmps_rt['total_lmp_rt'],
                         name = 'RT prices',
                         yaxis = 'y4',
                         line_shape='vh'
              ))

              
fig.update_layout(yaxis4=dict(  range=[0, 300],
                                title="Spot price [$/MWh]",
                                titlefont=dict(
                                    color="#9467bd"
                                ),
                                tickfont=dict(
                                    color="#9467bd"
                                ),
                                anchor="free",
                                overlaying="y",
                                side="right",
                                position=1
                             ))

fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=1.1, y=0),
                  width=900, height=350,
                  margin=dict(l=10, r=10, t=10, b=10))

fig.update_yaxes({'showgrid':True,
                  'gridwidth':1,
                  'gridcolor':'lightgray',
                  'linecolor':'black',
                  'mirror':True,
                  'showline':True})

fig.update_xaxes(showgrid=True,
                 gridwidth=1,
                 gridcolor='lightgray',
                 linecolor='black',
                 mirror = True,
                 showline = True,
                 rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1d", step="day", stepmode="backward"),
                        dict(count=14, label="1w", step="day", stepmode="backward"),
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=1, label="YTD", step="year", stepmode="todate"),
                        dict(count=1, label="1y", step="year", stepmode="backward"),
                        dict(step="all")
                    ])
                )
            )

fig.show()


# Compare the revenue from capacity and arbitrage

In [ ]:
db = 'storage_value_arbi_capacity_PJM.db'
table = 'storage_value'
cols = [col[0] for col in sm.list_columns(db, table)]
data = sm.get_data(cols, table, db)

from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = go.Figure()


for Zone in zone_list[1:2]:
    DATA = data[(data['arbi_type']=='DA+RT')&
                (data['zone']==Zone)&
                (data['rte']==40)][:-1]

    fig.add_trace(go.Bar(x=DATA['year'], y=DATA['SV_capa'],
                         name='capacpity')
                 )
    fig.add_trace(go.Bar(x=DATA['year'], y=DATA['SV_arbi'],
                         name='arbitrage')
                 )
    
    
fig.update_layout(title={'text': "SV (%s)"%(Zone),
                   'y':0.9,
                   'x':0.5,
                   'xanchor': 'center',
                   'yanchor': 'top'},
                    xaxis_title="Financial year",
                    yaxis_title="Net revenue [$/Year]",
                    xaxis=dict(
                        tickvals = DATA['year'].tolist()
                    ), barmode='stack')

fig.show()